<a href="https://colab.research.google.com/github/samp3209/capstone/blob/main/search_for_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Link google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

# Define the path of the folder containing the images
folder_path = '/content/drive/MyDrive/MassOutput/Test/'

# Create an empty list to store the file names of the images
file_names = []

# Loop through all the files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is an image (ends with .jpg, .jpeg, or .png)
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        # Get the file name of the image without the folder path
        file_name = os.path.basename(filename)
        # Append the file name to the list
        file_names.append(file_name)

# Print the file names of the images
print(file_names)

["chauncey the beaver,cartoon,blue,like he's holding something,looking at computer screen,in front of computer screen.png", 'chauncey the beaver,cartoon,green,reading,reading,in park.png', 'chauncey the beaver,cartoon,orange and blue,in motion,playing baseball,on field.png']


In [13]:
import os
from collections import defaultdict

# Define the path of the folder containing the images
folder_path = '/content/drive/MyDrive/MassOutput/Test/'

# Create a dictionary to store the file names and their descriptions
file_descriptions = defaultdict(str)

# Loop through all the files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is an image (ends with .jpg, .jpeg, or .png)
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        # Get the file name of the image without the folder path
        file_name = os.path.basename(filename)
        # Append the file name and description to the dictionary
        file_descriptions[file_name] = file_name.split('.')[0].split(', ')

# Define a function to calculate the Jaccard similarity between two sets
def jaccard_similarity(a, b):
    intersection = len(set(a).intersection(set(b)))
    union = len(set(a).union(set(b)))
    return intersection / union

# Define a query
query = 'chauncey'

# Create a dictionary to store the similarity scores of the files
similarity_scores = {}

# Loop through all the files in the dictionary
for file_name, file_description in file_descriptions.items():
    # Calculate the Jaccard similarity between the query and the file description
    similarity_scores[file_name] = jaccard_similarity(set(query.split()), set(file_description))

# Sort the files by their similarity scores in descending order
sorted_files = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)

# Print the top 3 files and their similarity scores
for file_name, score in sorted_files[:3]:
    print(file_name, ':', score)

chauncey the beaver,cartoon,blue,like he's holding something,looking at computer screen,in front of computer screen.png : 0.0
chauncey the beaver,cartoon,green,reading,reading,in park.png : 0.0
chauncey the beaver,cartoon,orange and blue,in motion,playing baseball,on field.png : 0.0
